In [6]:
import zipfile
from typing import *
import os
import json
import yaml

log_home = '../logs'

In [3]:
def get_log_file_line(file: str, level='DEBUG') -> Generator:
    if file.endswith('.log'):
        for line in open(file, 'r', encoding='utf-8'):
            if level in line:
                yield line
    elif file.endswith('.zip'):
        with zipfile.ZipFile(file, 'r') as zip:
            for zip_file in zip.namelist():
                file_bytes = zip.read(zip_file)
                for line in file_bytes.decode('utf-8').split('\n'):
                    if level in line:
                        yield line
                

In [8]:
rag_log_files = [os.path.join(log_home, file) for file in os.listdir(log_home) if file.startswith('rag')]

interesting_data = []

for rag_file in rag_log_files:
    for line in get_log_file_line(rag_file, level='DEBUG'):
        try:
            data = line.split('|')[-1].strip()
            data = eval(data)
            if data['intent']['name'] != 'others':
                interesting_data.append(data)
        except Exception:
            pass

In [10]:
story = {'stories': []}
for d in interesting_data:
    story['stories'].append({
        'message': d['query'],
        'intent': d['intent']['name']
    })

with open('../config/qq.story.yml', 'w', encoding='utf-8') as fp:
    yaml.dump(story, fp, allow_unicode=True)